<a href="https://colab.research.google.com/github/fllopes/https---github.com-fllopes-FUNCATE_FREL_2022/blob/master/GoogleColabMainCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Fiona
!pip install shapely
!pip install pyproj
!pip install geopandas
!pip install rasterio
!pip install rtree
!pip install pygeos

In [43]:
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
import matplotlib.pyplot as plt
from pathlib import Path
import datetime
from datetime import datetime
import pygeos
import rtree
# from datetime import date


class Script_Configs:
    
    def __init__(self):
    
        self.input_path = ''
    
        self.output_path = ''
        
        self.file_extention = '.shp'
        
        self.plot = True
        
    def input_pth( self, file_name ):
        
        return str( self.input_path + file_name )



# List all files in directory using pathlib
def locate_files(folder, extension):

    located_files = [entry for entry in Path(folder).iterdir() if entry.is_file() and entry.suffix == extension]

    return located_files



def main():
    
    log = process_log('main', 'Início')
        
    configs = Script_Configs()
    
    files = locate_files(configs.input_path, configs.file_extention)
    
    print('\nNumber of files located and listed for processing: ', len(files))
    
    trigger_algorithm(configs, files)
    
    log.close_log()



def trigger_algorithm(configs, files):
    
    log = process_log('trigger_algorithm', 'Runnig the algorithm on all the files sequencially.')
    
    for i, file in enumerate(files):
        
        print('\nProcessing file ({}/{} - {}%): '.format(i + 1, len(files), round((i + 1) * 100 / len(files), 1)), file.stem)
    
        input_data = gpd.read_file( file )
        
        print('  Load: success. ', str(datetime.now() - log.Start_time))


        input_data_exp = multipol_to_pol(input_data)

        print('  Multipolygon conversion to polygon: success. ', str(datetime.now() - log.Start_time))


        remove_dimension_z(input_data_exp)
        
        print('  Removal of dimension z: success. ', str(datetime.now() - log.Start_time))

        
        # diss = input_data_exp.dissolve(by = ['C_PRETORIG', 'C_PRETVIZI', 'CATEGORIG', 'CATEGVIZI', 'TIPO', 'CDW', 'CLITTER', 'CTOTAL4INV', 'CAGB', 'CBGB'], as_index=False)
        
        # export(diss, configs, file.stem)
                
        # print('  Dissolve: success. ', str(datetime.now() - log.Start_time))
        
    
        union = gpd.overlay(input_data_exp, input_data_exp, how = 'union', keep_geom_type = False)
        
        export(input_data_exp, '_union', configs, file.stem, log)
        
        print('  Union: success. ', str(datetime.now() - log.Start_time))
        
        
        natural_only = union[ union.TIPO_1 == 'NATURAL', union.TIPO_2 == 'NATURAL' ]

        antropic_only = union[ union.TIPO_1 == 'ANTROPIZADA', union.TIPO_2 == 'ANTROPIZADA']
        
        print('Natural only: ', len(natural_only), 'Antropic only: ', len(antropic_only))
    
    
        if configs.plot: plot_data(diss)
        
        else: pass
    
        break

    log.close_log()



def multipol_to_pol(data):
    
    distinct_types = geometry_type_check(data)
        
    if 'MultiPolygon' in distinct_types:
    
        new_data = data.explode()
            
        multipol_to_pol(new_data)

    else: new_data = data
        
    return new_data



def geometry_type_check(data):
    
    distinct_types = []

    for pol in data.iterfeatures():
        
        if pol['geometry']['type'] not in distinct_types:
            
            distinct_types.append(pol['geometry']['type'])
            
        else: pass
    
    print(distinct_types)
    
    return distinct_types



def remove_dimension_z(data):   

    new_geo = []
    
    for pol in data.geometry:
        
        if pol.has_z:
            
            if pol.geom_type == 'Polygon':
                                
                lines = [xy[:2] for xy in list(pol.exterior.coords)]
                
                new_p = Polygon(lines)
                
                new_geo.append(new_p)
                
    data.geometry = new_geo    



def export(data_to_export, operation, configs, original_file_name, log):

    data_to_export.to_file(str(configs.output_path + original_file_name + operation + '.shp'), driver = 'ESRI Shapefile')
    
    print('Export: success', str(datetime.now() - log.Start_time))



def plot_data(*args):
    
    fig, ax = plt.subplots(figsize = (10,8))
    
    for data in args:
        
        data.plot(ax = ax)



class process_log:

    def __init__(self, process, description):
        
        self.Process = process
        self.Description = description
        self.Start_time = datetime.now()

        print(str('\n[Log] Processo "' + self.Process + '": ' + self.Description + '\n' + '[Log] Início "' + self.Process + '": ' + str(self.Start_time)))

    def close_log(self):

        end_time = datetime.now()

        print(str('[Log] Fim "' + self.Process + '": ' + str(end_time) + '\n        Duração: ' + str(datetime.now() - self.Start_time)))



main()





[Log] Processo "main": Início
[Log] Início "main": 2022-10-27 17:01:01.287560

Number of files located and listed for processing:  1

[Log] Processo "trigger_algorithm": Runnig the algorithm on all the files sequencially.
[Log] Início "trigger_algorithm": 2022-10-27 17:01:01.289643

Processing file (1/1 - 100.0%):  amazonia_carbono_vegetacao_1
  Load: success.  0:00:00.727317
['Polygon', 'MultiPolygon']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:111: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.


['Polygon']
  Multipolygon conversion to polygon: success.  0:00:03.304623
  Removal of dimension z: success.  0:00:04.036879


ImportError: ignored

In [36]:
!pip install pygeos


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 4.2 MB/s 
